In [1]:
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q gradio
!pip install -q openai
!pip install -q gTTS
!pip install elevenlabs
!pip install pygame

  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      Checking for Rust toolchain....
      
      Cargo, the Rust package manager, is not installed or is not on PATH.
      This package requires Rust and Cargo to compile extensions. Install it through
      the system's package manager or via https://rustup.rs/
      
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 1.4 MB/s eta 0:00:0000:0100:01


In [2]:
!apt-get install -y portaudio19-dev

zsh:1: command not found: apt-get


In [4]:
import whisper
import gradio as gr 
import time
import warnings
import json
import openai
import os
import pygame
from pydub import AudioSegment
from pydub.playback import play
import io
from gtts import gTTS
from elevenlabs import *
from elevenlabs import save, generate, set_api_key

ModuleNotFoundError: No module named 'gradio'

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
openai.api_key = '<insert openai API key here'

from elevenlabs import set_api_key
set_api_key("<insert elevenlabs api key>")

In [ ]:
model = whisper.load_model("base")

100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 128MiB/s]


In [ ]:
model.device

device(type='cuda', index=0)

In [ ]:
!ffmpeg -f lavfi -i anullsrc=r=44100:cl=mono -t 10 -q:a 9 -acodec libmp3lame Temp.mp3

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

In [ ]:
def speech_to_text(audio):
  
    language = 'en'

    audio = whisper.load_audio(audio)
    audio = whisper.pad_or_trim(audio)

    mel = whisper.log_mel_spectrogram(audio).to(model.device)

    _, probs = model.detect_language(mel)

    options = whisper.DecodingOptions()
    result = whisper.decode(model, mel, options)
    result_text = result.text
    return result_text

In [ ]:
def chatgpt_api(input_text):

        chat_completion = openai.ChatCompletion.create(
              model="gpt-3.5-turbo",
              messages= input_text,
              temperature=.5,
              max_tokens=100,
              top_p=1,
              frequency_penalty=0,
              presence_penalty=0
              )

        reply = chat_completion.choices[0].message.content
        
        return reply

In [ ]:
def text_to_speech(out_result, language = 'en'):

    audio = generate(
        text=out_result,
        voice="Rachel",
        model="eleven_monolingual_v1"
        )

    save(
      audio,
      'Temp.mp3'
    ) 
    
    return "Temp.mp3"

In [ ]:
# Instantiate conversation_history
# Set initial prompt
conversation_history = [{"role": "system", "content": "You are a sassy and acid conversational chatbot. Keep the conversation going asking questions back. Begin every answer with 'bruh'"}]

In [ ]:
def transcribe(audio):

    global conversation_history

    # Extract question text from voice
    question_text = speech_to_text(audio)

    # Append question_text to conversation_history
    conversation_history.append(
            {"role": "user", "content": question_text},
        )

    # Feed ChatOpenAI with conversation_history
    answer_text = chatgpt_api(conversation_history)

    # Synthetize voice
    voice_answer = text_to_speech(answer_text)

    # Append answer_text to conversation_history
    conversation_history.append(
            {"role": "assistant", "content": answer_text},
        )


    return [question_text, answer_text, voice_answer]

In [ ]:
output_1 = gr.Textbox(label="Speech to Text")
output_2 = gr.Textbox(label="ChatGPT Output")
output_3 = gr.Audio("Temp.mp3")

gr.Interface(
    title = 'OpenAI Whisper and ChatGPT ASR Gradio Web UI', 
    fn=transcribe, 
    inputs=[
        gr.inputs.Audio(source="microphone", type="filepath")
    ],

    outputs=[
        output_1,  output_2, output_3
    ],
    live=True).launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>